In [ ]:
!pip install transformers torch spotipy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.9/268.9 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
#  Step 2: Import Libraries
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import random


In [ ]:
#  Step 3: Set Your Spotify Credentials Securely

os.environ['SPOTIPY_CLIENT_ID'] = 'cf49572814e544c9a6bcaf5978b3dd86'
os.environ['SPOTIPY_CLIENT_SECRET'] = '0e69e658798c405daaa866101166cb49'

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=os.getenv('SPOTIPY_CLIENT_ID'),
    client_secret=os.getenv('SPOTIPY_CLIENT_SECRET')
))


In [ ]:
#  Step 4: Load GPT-2 Large Model and Tokenizer
model_name = "gpt2-large"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

# Set pad_token to eos_token to avoid padding errors
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
#  Step 5: Prompt Builder Function
def build_prompt(mood, genre, keywords):
    keyword_str = ', '.join(keywords)
    prompt = (
        f"Write a {genre} song with a {mood} mood.\n"
        f"Include the following keywords: {keyword_str}.\n"
        f"Make the lyrics rhyming and poetic.\n\n"
        f"Verse:\n"
    )
    return prompt


In [ ]:
#  Step 6: Generate Lyrics using GPT-2
def generate_lyrics(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors='pt', padding=True)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=input_ids.shape[1] + max_length,
        temperature=0.9,
        top_k=50,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated[len(prompt):].strip().split('\n')[:8]


In [ ]:
#  Step 7: Search Spotify for Similar Songs
def search_spotify_song(keywords, limit=3):
    random.shuffle(keywords)
    query = " ".join(keywords)
    results = sp.search(q=query, type='track', limit=limit)
    songs = []
    for track in results['tracks']['items']:
        song_info = {
            "title": track['name'],
            "artist": track['artists'][0]['name'],
            "url": track['external_urls']['spotify']
        }
        songs.append(song_info)
    return songs


In [ ]:
#  Step 8: User Input and Execution
mood = input("Enter the mood (e.g., happy, sad, romantic): ")
genre = input("Enter the genre (e.g., pop, rap, rock): ")
keywords_input = input("Enter keywords separated by commas: ")
keywords = [k.strip() for k in keywords_input.split(',')]

prompt = build_prompt(mood, genre, keywords)
lyrics = generate_lyrics(prompt)

print("\n🎤 Generated Song Lyrics:\n")
for line in lyrics:
    print(line)

print("\n🎧 Similar Songs from Spotify:\n")
songs = search_spotify_song(keywords)
for song in songs:
    print(f"🎶 {song['title']} by {song['artist']}")
    print(f"🔗 Link: {song['url']}\n")


Enter the mood (e.g., happy, sad, romantic): happy
Enter the genre (e.g., pop, rap, rock): pop
Enter keywords separated by commas: weekend, party, friends

🎤 Generated Song Lyrics:

My name is Joe and I'm here with my family

I'm going to do you a favor

Tell me how it's going to be

Now come on home!


🎧 Similar Songs from Spotify:

🎶 Weekend! Party With Your Friends!!! by Kara's Walk Home
🔗 Link: https://open.spotify.com/track/5JJVm9lFMZzPKMfPL2VfQb

🎶 3005 by Childish Gambino
🔗 Link: https://open.spotify.com/track/2X6b7zLdIxCejd6GqVcQ9M

🎶 Weekend Friend by Goth Babe
🔗 Link: https://open.spotify.com/track/2d3QlXE6FXFDeodiS66yjM

